In [ ]:
import cv2
#from imutils import face_utils
import pandas as pd
import numpy as np
import mediapipe as mp

import os
import re

#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score

#import joblib

In [ ]:
# Пути до ресурсов
DATASET_BINARY_DATA = r"..\binary_data/"
DATASET_ALL_DATA = r"..\all_data/"
OUTPUT_LANDMARKS_FOLDER = r"..\output_landmarks/"

In [ ]:
# Загружаем модель
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

In [ ]:
def write_output_landmarks_file(image_name, label, landmarks):
    image_name = os.path.splitext(image_name)[0]
    # Папка для сохранения txt-файлов
    output_dir = OUTPUT_LANDMARKS_FOLDER
    os.makedirs(output_dir, exist_ok=True)

    # Путь к файлу, куда сохранить координаты
    output_path = os.path.join(output_dir, f'{image_name}.txt')

    # Сохраняем в файл
    with open(output_path, 'w') as f:
        f.write(f'{label}\n')
        for x, y in landmarks:
            f.write(f'{x} {y}\n')


In [ ]:
def normalize_landmarks(points):
    points = points.astype(np.float32)
    #points = landmarks.reshape(-1, 2)
    left_eye = points[33]
    right_eye = points[263]

    center_x = (left_eye[0] + right_eye[0]) / 2
    center_y = (left_eye[1] + right_eye[1]) / 2

    points[:, 0] -= center_x
    points[:, 1] -= center_y

    eye_dist = np.linalg.norm(left_eye - right_eye)
    if eye_dist > 0:
        points = points / eye_dist
    return points

def get_landmarks(image_path, landmarks = []):
    image_array = np.fromfile(image_path, dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError("Изображение не загружено!")
            
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    height, width = image_rgb.shape[:2]

    # Детектируем лицо
    faces = face_mesh.process(image_rgb)

    if not faces.multi_face_landmarks:
        return
        #raise ValueError("Лицо не найдено на изображении")

    shape = faces.multi_face_landmarks[0]

    landmarks = np.array([(p.x * width, p.y * height) for p in shape.landmark])
    #landmarks = np.concatenate((np.concatenate((landmarks[pack_identify["first"]], landmarks[pack_identify["second"]]), axis=0), landmarks[pack_identify["third"]]), axis=0).flatten()

    return landmarks

In [ ]:
def numeric_key(name):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', name)]

def build_dataframe(dataset_dir, all_landmarks, all_labels, get_label_func = lambda k : k // 8): # получаем числовой код папки
    k = 0
    for label_dir in sorted(os.listdir(dataset_dir), key=numeric_key):
        label_path = os.path.join(dataset_dir, label_dir)
        if not os.path.isdir(label_path):
            continue

        for filename in os.listdir(label_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(label_path, filename)

                landmarks = get_landmarks(image_path)
                if landmarks is None: continue
                label = get_label_func(k) % 2
                
                all_landmarks.append(landmarks.flatten())
                all_labels.append(label)

                write_output_landmarks_file(filename, label, landmarks)

        k += 1

In [ ]:
all_landmarks = []
all_labels = []

build_dataframe(DATASET_ALL_DATA, all_landmarks, all_labels) # получаем числовой код 

In [ ]:
columns = []
for idx in list(range(0, 468)):
    columns.append(f"x{idx}")
    columns.append(f"y{idx}")

df = pd.DataFrame(all_landmarks, columns=columns)
df['label'] = all_labels
df = df.sample(frac=1).reset_index(drop=True)

print(df["label"].unique())
print(df["label"].value_counts())

df

In [ ]:
df.to_csv("data.csv", sep=';', index=False)

In [ ]:
# Разделим данные
#

# X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(
#     df_binary.drop(columns=['label']),  # все колонки кроме 'label'
#     df_binary['label'],                 # сами метки
#     test_size=0.2,                # 20% на тест
#     stratify=df_binary['label'],         # сбалансированная разбивка по классам
#     random_state=42
# )

# Обучим модель
# model_subtype = LogisticRegression(max_iter=1000)
# model_subtype.fit(X_train_subtype, y_train_subtype)

# model_binary = LogisticRegression(max_iter=1000)
# model_binary.fit(X_train_binary, y_train_binary)

# # Оценим качество
# # y_pred_subtype = model_subtype.predict(X_test_subtype)
# # print(f"Accuracy subtype: {accuracy_score(y_test_subtype, y_pred_subtype):.4f}")

# y_pred_binary = model_binary.predict(X_test_binary)
# print(f"Accuracy binary class: {accuracy_score(y_test_binary, y_pred_binary):.4f}")

In [ ]:
# def predict_dichotomy(image_path, model):
#     landmarks = get_landmarks(image_path)
    
#     X_input = pd.DataFrame([landmarks], columns=columns)

#     prediction = model.predict(X_input)[0]
#     proba = model.predict_proba(X_input)[0]

#     return prediction, proba

In [ ]:
# joblib.dump(model_subtype, 'subtype_classifier.pkl')

# joblib.dump(model_binary, 'logic_ethics_classifier.pkl')

# subtypes = {
#     0 : "Шизоидный",
#     1 : "Параноидальный",
#     2 : "Нарциссический",
#     3 : "Психопатический",
#     4 : "Компульсивный",
#     5 : "Истерический",
#     6 : "Депрессивный",
#     7 : "Мазохистический"
# }

In [ ]:
# model_subtype_file = joblib.load("subtype_classifier.pkl")  # путь к сохранённой модели
# model_binary_file = joblib.load("logic_ethics_classifier.pkl")  # путь к сохранённой модели

In [ ]:
# # 2. Инициализация детектора и предиктора
# result_subtype, confidence_subtype = predict_dichotomy("../all_data/12/16 (2).jpg", model_subtype_file)
# print(f"Подтип личности: {subtypes[result_subtype]} (Уверенность: {max(confidence_subtype):.2f})")

# result_binary, confidence_binary = predict_dichotomy("../all_data/12/16 (2).jpg", model_binary_file)
# print(f"Дихотомия: {'Этика' if result_binary == 0 else 'Логика'} (Уверенность: {max(confidence_binary):.2f})")


In [ ]:
# def normalize_landmarks(landmarks):
#     """
#     landmarks: np.array с shape (N, 2), координаты в пикселях
#     Возвращает: нормализованные landmarks и параметры центра и масштаба
#     """
#     left_eye = landmarks[33]
#     right_eye = landmarks[263]
#     center = (left_eye + right_eye) / 2
#     scale = np.linalg.norm(left_eye - right_eye)

#     normalized = (landmarks - center) / scale
#     return normalized, center, scale


# def denormalize_landmarks(normalized, center, scale):
#     """
#     Восстанавливает нормализованные точки обратно в пиксельные координаты
#     """
#     return (normalized * scale) + center

# def draw_normalized_landmarks(image_path):
#     # 1. Загрузка изображения
#     image = cv2.imread(image_path)
#     if image is None:
#         raise ValueError("Не удалось загрузить изображение")
#     h, w = image.shape[:2]

#     # 2. Инициализация MediaPipe
#     mp_face_mesh = mp.solutions.face_mesh
#     face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

#     # 3. Обработка изображения
#     rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     results = face_mesh.process(rgb)

    # 4. Проверка наличия лица
    # if not results.multi_face_landmarks:
    #     print("Лицо не найдено")
    #     return

    # # 5. Извлечение точек и нормализация
    # face_landmarks = results.multi_face_landmarks[0]
    # landmarks = np.array([
    #     [lm.x * w, lm.y * h]
    #     for lm in face_landmarks.landmark
    # ])
    # normalized, center, scale = normalize_landmarks(landmarks)
    # restored = denormalize_landmarks(normalized, center, scale)

    # # 6. Отображение денормализованных точек
    # for point in restored.astype(int):
    #     x, y = point
    #     cv2.circle(image, (x, y), radius=1, color=(0, 255, 255), thickness=-1)

    # # 7. Отображение изображения
    # cv2.imshow("Normalized landmarks (restored)", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [ ]:
# draw_normalized_landmarks("../all_data/12/16 (2).jpg")